
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

<i18n value="2630af5a-38e6-482e-87f1-1a1633438bb6"/>

# AutoML

Databricksの <a href="https://docs.databricks.com/applications/machine-learning/automl.html" target="_blank">AutoML</a> を使って、UIもしくはプログラムから機械学習モデルを自動的に構築することができます。モデル学習のためにデータセットを準備し、（HyperOptを使用して）複数のモデルを作成、チューニング、評価する一連の試行を実行して記録します。 

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png)このレッスンでは、以下のことを行います。<br>
 - AutoMLを使用してモデルを自動的にトレーニングおよびチューニングする
 - PythonとUIでAutoMLを実行する
 - AutoMLの実行結果を解釈する

In [0]:
%run "./Includes/Classroom-Setup"

<i18n value="7aa84cf3-1b6c-4ba4-9249-00359ee8d70a"/>

現在、AutoMLはXGBoostとsklearn（シングルノードモデルのみ）を使用しており、それぞれでハイパーパラメータを最適化しています。

In [0]:
file_path = f"{DA.paths.datasets}/airbnb/sf-listings/sf-listings-2019-03-06-clean.delta/"
airbnb_df = spark.read.format("delta").load(file_path)
train_df, test_df = airbnb_df.randomSplit([.8, .2], seed=42)

<i18n value="1b5c8a94-3ac2-4977-bfe4-51a97d83ebd9"/>

AutoMLを使って最適な <a href="https://docs.databricks.com/applications/machine-learning/automl.html#regression" target="_blank">回帰</a> モデルを探索できます。 

必要なパラメータ
* **`dataset`** \- 学習用の特徴量とターゲット値を格納したSpark DataFrame または Pandas DataFrame を用意します。Spark DataFrameを使う場合、.toPandas()を呼び出すことで内部でPandas DataFrameに変換します。データ量が多い場合にはOOM (out of memory)にならないように注意してください。
* **`target_col`** \- ターゲットラベルのカラム名

また、オプションのパラメータとして次を指定できます。
* **`primary_metric`** \- 最適なモデルを選択するために優先的に使用するメトリック。各試行ではいくつかのメトリックを計算しますが、このprimary\_metricによって最適なモデルを決定します。回帰に対していは、次のいずれかを指定します：  **`r2`** (R squared - デフォルト), **`mse`** (mean squared error : 平均二乗誤差), **`rmse`** (root mean squared error : 二乗平均平方根誤差), **`mae`** (mean absolute error : 平均絶対誤差)。
* **`timeout_minutes`** \- AutoML の試行が完了するまでの最大待ち時間。 **`timeout_minutes=None`** と指定するとタイムアウトの制約を受けずに試行を実行します。
* **`max_trials`** \- 実行する試行回数の最大値。 **`max_trials=None`** と指定すると、完了するまで実行します。(Databricks Runtime 10.3 MLからは、max_trialsが廃止されたため、設定しても無効になります。 )

In [0]:
from databricks import automl

summary = automl.regress(train_df, target_col="price", primary_metric="rmse", timeout_minutes=5, max_trials=10)

2024/05/06 09:26:29 WARNING databricks.automl.client.validation: Parameter max_trials is deprecated and has no effect. This parameter will be removed in a future Databricks Runtime release. Use timeout_minutes to control the duration of an AutoML experiment. AutoML will automatically stop tuning models if the validation metric no longer improves.
2024/05/06 09:26:30 INFO databricks.automl.client.manager: AutoML will optimize for root mean squared error metric, which is tracked as val_root_mean_squared_error in the MLflow experiment.
2024/05/06 09:26:32 INFO databricks.automl.client.manager: MLflow Experiment ID: 3529245064622630
2024/05/06 09:26:32 INFO databricks.automl.client.manager: MLflow Experiment: https://adb-6408635526599659.19.azuredatabricks.net/?o=6408635526599659#mlflow/experiments/3529245064622630
2024/05/06 09:28:12 INFO databricks.automl.client.manager: Data exploration notebook: https://adb-6408635526599659.19.azuredatabricks.net/?o=6408635526599659#notebook/3529245064

,Train,Validation,Test
root_mean_squared_error,219.833,328.906,383.472
score,0.420,0.240,0.126
mean_squared_error,48326.380,108179.120,147050.781
example_count,3466.000,1169.000,1151.000
mean_on_target,210.863,219.494,224.251
r2_score,0.420,0.240,0.126
mean_absolute_error,61.889,79.632,86.697
mean_absolute_percentage_error,0.296,0.350,0.374
max_error,8782.202,7384.221,8532.636
sum_on_target,730852.000,256588.000,258113.000


<i18n value="57d884c6-2099-4f34-b840-a4e873308ffe"/>

前のセルを実行すると、2つのノートブックと1つのMLflow実験が表示されます。
* **`Data exploration notebook (データ探索ノートブック)`** \- 入力列ごとの値、頻度とその他の情報をまとめたデータ概要レポート。
* **`Best trial notebook (最適試行ノートブック`** \- AutoMLによって構築されたベストモデルを再現するためのソースコード。
* **`MLflow experiment`** \- ArtifactのRootロケーション、experiment(実験)のID、タグなどのハイレベルな情報が含まれています。実行（Run）一覧には、ノートブックやモデルの場所、トレーニングパラメータ、総合的なメトリクスなど、各トライアルに関する詳細なサマリーが記載されています。

このノートブックとMLflowの実験を掘り下げると、何がわかるでしょうか？

さらに、AutoMLは、最適な試行から得られたメトリックの短いリストを表示します。

In [0]:
print(summary.best_trial)


Model: Pipeline
Model path: dbfs:/databricks/mlflow-tracking/3529245064622630/b96e349e97de43c999dd1fd2ab5df9f8/artifacts/model
Preprocessors: [('boolean', Pipeline(steps=[('cast_type',
         FunctionTransformer(func=<function <lambda> at 0x7f2fc0d0a950>)),
        ('imputers',
         ColumnTransformer(remainder='passthrough', transformers=[])),
        ('onehot',
         OneHotEncoder(drop='first', handle_unknown='ignore'))]), ['review_scores_checkin_na', 'review_scores_accuracy_na', 'review_scores_value_na', 'review_scores_cleanliness_na', 'bathrooms_na', 'beds_na', 'bedrooms_na', 'rev...
Training duration: 0.323 minutes
Evaluation metric score: 328.906
Evaluation metric: root mean squared error (tracked as val_root_mean_squared_error)



<i18n value="3c0cd1ec-8965-4af3-896d-c30938033abf"/>

さて、AutoMLから取得したモデルをテストデータに対してテストすることができます。<a href="https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.spark_udf" target="_blank">mlflow.pyfunc.spark\_udf</a> を使って、モデルをUDFとして登録し、テストデータに対して並列に適用していきます。

In [0]:
# Load the best trial as an MLflow Model
import mlflow

model_uri = f"runs:/{summary.best_trial.mlflow_run_id}/model"

predict = mlflow.pyfunc.spark_udf(spark, model_uri)
preds = predict(*test_df.drop("price").columns)
pred_df = test_df.withColumn("prediction", preds[0])
display(pred_df)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="rmse")
rmse = regression_evaluator.evaluate(pred_df)
print(f"RMSE on test dataset: {rmse:.3f}")

RMSE on test dataset: 113.782


&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>